<a href="https://colab.research.google.com/github/deklismam/portfolio_datascience/blob/codenation/second_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Teste Codenation - AceleraDev 05.04.2020

Read.me
# Descubra as melhores notas de matemática do ENEM 2016

Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016. 

## Tópicos

Neste desafio você aprenderá:

- Lógica
- Análise de dados
- Estatística
- Regression

## Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

Muitas universidades brasileiras utilizam o ENEM para selecionar seus futuros alunos e alunas. Isto é feito com uma média ponderada das notas das provas de matemática, ciências da natureza, linguagens e códigos, ciências humanas e redação.

- matemática: 3
- ciências da natureza: 2
- linguagens e códigos: 1.5
- ciências humanas: 1
- redação: 3

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna **NU_NOTA_MT**) de quem participou do ENEM 2016. Poste-a em ordem decrescente.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: **NU_INSCRICAO** e **NU_NOTA_MT**.

Faça o upload do arquivo answer.csv usando o botão "Submeter resposta".

## Observações

O objetivo do desafio é avaliar a sua capacidade de resolver problemas, então ele pode ser resolvido em qualquer linguagem de programação ou ferramenta que você tenha facilidade. Mas recomendamos o uso de linguagens com maior ferramental de matemática e estatística como Python, R, Scala ou Julia.


## O plano de solução

No arquivo train.csv há os dados dos candidatos, inclusas as notas de matemática "NU_NOTA_MT" e o arquivo test.csv há os dados dos candidatos sem o parâmetro "NU_NOTA_MT".

O plano aqui é utilizar Machine Learning para calcular as notas de matemática para os candidatos presentes em test.csv com base nos dados de train.csv

Mais especificamente, utilizaremos Regressão Linear Múltipla, onde consideramos que a nota de matemática é relacionadas às demais notas conforme a fórmula:

NU_NOTA_MT = a*NU_NOTA_CN + b*NU_NOTA_LC + c*NU_NOTA_CH + d*NU_NOTA_REDACAO + e

Onde:
* a, b, c, d, e: são os coeficientes a serem calculados pela Regressão Linear Múltipla

Obtendo estes parâmetros, aplicaremos a fórmula no dataframe test.csv e, assim, calcularemos a nota de matemática de cada inscrito.

In [1]:
# importando os pacotes necessários 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
from google.colab import files

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# realizando o upload de arquivo já baixado no computador. Caso necessário, segue o link para download do arquivo postado no desafio: https://www.dropbox.com/s/8eqdylw320kcdyd/train.csv?dl=0
n=files.upload()
df = pd.read_csv(io.BytesIO(n['train.csv']))

Saving train.csv to train.csv


In [5]:
# veremos o tamanho do arquivo (linhas) e os parâmetros (colunas) do dataframe
print("Parâmetros: \t {}".format(df.shape[1]))
print("variáveis: \t {}".format(df.shape[0]))
display(df.dtypes)


Parâmetros: 	 167
variáveis: 	 13730


Unnamed: 0                  int64
NU_INSCRICAO               object
NU_ANO                      int64
CO_MUNICIPIO_RESIDENCIA     int64
NO_MUNICIPIO_RESIDENCIA    object
                            ...  
Q046                       object
Q047                       object
Q048                       object
Q049                       object
Q050                       object
Length: 167, dtype: object

In [6]:
# ver como são apresentados os dados no dataframe, nas cinco primeiras linhas
df.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,...,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,1,1,1,1,1,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,1,2304400.0,Fortaleza,23.0,CE,3,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,5,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,1,3304557.0,Rio de Janeiro,33.0,RJ,1,9,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,B,B,A,B,A,B,C,A,B,D,B,C,B,C,F,C,2.0,0.0,5.0,5.0,5.0,5,5,5,5,2,5,5,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,1,1505304.0,Oriximiná,15.0,PA,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,A,A,B,A,A,B,A,A,A,B,F,D,5.0,4.0,3.0,5.0,4.0,5,5,4,3,1,4,5,NaN,A,A,A,A,A,A,A,A,A


In [0]:
# excluindo aqueles candidatos que faltaram ('0') ou foram eliminados ('2'), já que suas notas apenas influenciarão o cálculo dos coeficientes.
df_clean = df.copy()
df_clean.drop(df_clean[df_clean.TP_PRESENCA_CN == 0].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.TP_PRESENCA_CN == 2].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.TP_PRESENCA_LC == 0].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.TP_PRESENCA_LC == 2].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.TP_PRESENCA_CH == 2].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.TP_PRESENCA_CH == 0].index, axis=0, inplace=True)

In [8]:
# verificando quantas variáveis restaram
print("Parâmetros: \t {}".format(df_clean.shape[1]))
print("variáveis: \t {}".format(df_clean.shape[0]))
display(df_clean.dtypes)

# ou seja, faltaram ou foram eliminados na prova 3633 inscritos.

Parâmetros: 	 167
variáveis: 	 10097


Unnamed: 0                  int64
NU_INSCRICAO               object
NU_ANO                      int64
CO_MUNICIPIO_RESIDENCIA     int64
NO_MUNICIPIO_RESIDENCIA    object
                            ...  
Q046                       object
Q047                       object
Q048                       object
Q049                       object
Q050                       object
Length: 167, dtype: object

In [9]:
# excluido as colunas desnecessárias para a Regressão, ou seja, mantendo as seguintes colunas NU_INSCRICAO, NU_NOTA_MT, NU_NOTA_CN, NU_NOTA_LC, NU_NOTA_CH, NU_NOTA_REDACAO
df2_clean = df_clean.loc[:,['NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_REDACAO']]

#observar como estão apresentados os dados no novo dataframe
df2_clean.head()

,NU_NOTA_MT,NU_NOTA_CN,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_REDACAO
0,399.4,436.3,581.2,495.4,520.0
1,459.8,474.5,599.0,544.1,580.0
5,364.5,439.7,410.9,583.2,620.0
6,529.2,420.1,484.5,604.2,560.0
7,566.7,619.6,611.2,625.8,620.0


In [10]:
# aqui está outra forma de remover os candidatos sem notas em quaisquer dos parâmetros utilizados na Regressão

df2_clean.dropna(axis=0, how='any')
print("Parâmetros: \t {}".format(df2_clean.shape[1]))
print("variáveis: \t {}".format(df2_clean.shape[0]))
display(df2_clean.dtypes)
df2_clean.head()

#observe que as variáveis continuam com a mesma quantidade quando removemos os candidatos que faltaram ou foram eliminados. Ou seja, no passo anterior conseguimos remover todos aqueles que não têm as quatro notas consideradas na Regressão

Parâmetros: 	 5
variáveis: 	 10097


NU_NOTA_MT         float64
NU_NOTA_CN         float64
NU_NOTA_LC         float64
NU_NOTA_CH         float64
NU_NOTA_REDACAO    float64
dtype: object

,NU_NOTA_MT,NU_NOTA_CN,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_REDACAO
0,399.4,436.3,581.2,495.4,520.0
1,459.8,474.5,599.0,544.1,580.0
5,364.5,439.7,410.9,583.2,620.0
6,529.2,420.1,484.5,604.2,560.0
7,566.7,619.6,611.2,625.8,620.0


In [11]:
# importar pacotes necessários para aplicação da Regressão Linear Múltipla

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# configurar notebook

sns.set_style

# determinando x e y

x = df2_clean.drop('NU_NOTA_MT', axis=1)
y = df2_clean['NU_NOTA_MT']
#criar modelo de Regressão Linear Múltipla
rlm_model = LinearRegression()
rlm_model.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
# imprimir os coeficientes
print("coeficiente: \t", rlm_model.coef_)
print("intercepto: \t", rlm_model.intercept_)

coeficiente: 	 [0.53698843 0.20774178 0.24262847 0.02064483]
intercepto: 	 -18.777348438076785


Ou seja:
* a = 0.53698843;
* b = 0.20774178
* c = 0.24262847
* d = 0.02064483
* e = -18.777348438076785